In [2]:
import rawpy
import imageio
import torch
import matplotlib.pyplot as plt
import cv2
import numpy as np

path = r"C:/Users/Revi/Desktop/Sony/sony_train/DSC01118.ARW"
with rawpy.imread(path) as raw:
    #rgb = raw.postprocess()
    data = raw.raw_image.copy()
    print(data[0,0])
    #print(raw.raw_image)
    #print(len(raw.raw_image))
    rgb = raw.postprocess()
    print(rgb[0,0])
    imageio.imsave('C:/Users/Revi/Desktop/Sony/aaa.jpeg', rgb)
    
    #print(raw.rgb_xyz_matrix)

800
[ 0 77  0]


In [2]:
width = 5496
height = 3672

with open("C:/Users/Revi/Desktop/Sony/sony_train/DSC01118.ARW", "rb") as rawimg:
    # Read the raw image as uint8
    bayer_im = np.fromfile(rawimg, np.uint8, width * height).reshape(height, width)

bayer_im[0,0]


#https://cs.brown.edu/courses/csci1290/labs/lab_raw/index.html

73

In [3]:
raw_filename = 'C:/Users/Revi/Desktop/Sony/sony_train/DSC01375.ARW'
 
with rawpy.imread(raw_filename) as raw:
    print(f'raw pattern:                  {raw.raw_pattern.tolist()}')          # decribes the pattern of the Bayer sensor
    print(f'CTS Matrix:')
    print(raw.rgb_xyz_matrix)       # camera specific XYZ to camara RGB conversion matrix
    print('Color description: ', raw.color_desc)

raw pattern:                  [[0, 1], [3, 2]]
CTS Matrix:
[[ 1.0315 -0.439  -0.0937]
 [-0.4859  1.2734  0.2365]
 [-0.0734  0.1537  0.5997]
 [ 0.      0.      0.    ]]
Color description:  b'RGBG'


In [1]:
#https://www.numbercrunch.de/blog/2020/12/from-numbers-to-images-raw-image-processing-with-python/

def pack_avg_cst(raw_filename):
    with rawpy.imread(raw_filename) as raw:
        # Get raw image data
        image = np.array(raw.raw_image, dtype=np.double)

        # Get the raw pattern of the photo
        n_colors = raw.num_colors
        colors = np.frombuffer(raw.color_desc, dtype=np.byte)
        pattern = np.array(raw.raw_pattern)
        i_0 = np.where(colors[pattern] == colors[0])
        i_1 = np.where(colors[pattern] == colors[1])
        i_2 = np.where(colors[pattern] == colors[2])

        # Pack image and average the green channels
        image_packed = np.empty((image.shape[0]//2, image.shape[1]//2, n_colors))
        
        image_packed[:, :, 0] = image[i_0[0][0]::2, i_0[1][0]::2]
        image_packed[:, :, 1]  = (image[i_1[0][0]::2, i_1[1][0]::2] + image[i_1[0][1]::2, i_1[1][1]::2]) / 2
        image_packed[:, :, 2]  = image[i_2[0][0]::2, i_2[1][0]::2]

        print(image[0,3])
        print(image[1,2])
        print(image_packed[0,1])

        # CST Conversion
        cst = np.array(raw.rgb_xyz_matrix[0:3, :], dtype=np.double)
        sRGB_to_XYZ = np.array([[0.4124564, 0.3575761, 0.1804375],
                                [0.2126729, 0.7151522, 0.0721750],
                                [0.0193339, 0.1191920, 0.9503041]], dtype=np.double)
        sRGB_to_cam = np.dot(cst, sRGB_to_XYZ)
        norm = np.tile(np.sum(sRGB_to_cam, 1), (3, 1)).transpose()
        sRGB_to_cam = sRGB_to_cam / norm
        cam_to_sRGB = np.linalg.inv(sRGB_to_cam)
        
        image_sRGB = np.einsum('ij,...j', cam_to_sRGB, image_packed)  # performs the matrix-vector product for each pixel
        # apply sRGB gamma curve
        i = image_sRGB < 0.0031308
        j = np.logical_not(i)
        image_sRGB[i] = 323 / 25 * image_sRGB[i]
        image_sRGB[j] = 211 / 200 * image_sRGB[j] ** (5 / 12) - 11 / 200
        image_sRGB = np.clip(image_sRGB, 0, 1)

        plt.axis('off')
        plt.imshow(image_sRGB)
            
        
        #image_XYZ = torch.from_numpy(image_XYZ).unsqueeze(0)
        #return image_XYZ #Returns Tensor with size [1,H,W,C]

prep_image = pack_avg_cst('C:/Users/Revi/Desktop/Sony/sony_train/DSC01375.ARW')


NameError: name 'rawpy' is not defined

In [5]:
import torch.nn.functional as F

prep_image = pack_avg_cst('C:/Users/Revi/Desktop/Sony/sony_train/DSC01375.ARW')
print(prep_image.shape)

def resize_256(image):
    resized_256 = image.permute(0, 3, 1, 2) 
    resized_256 = F.interpolate(resized_256, size=(256, 256), mode='bilinear')
    return resized_256


resized_image = resize_256(prep_image)
print(resized_image.shape)

1012.0
992.0
[ 864. 1002.  944.]
torch.Size([1, 1836, 2752, 3])
torch.Size([1, 3, 256, 256])


In [143]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.ops import MLP
class i2p(nn.Module):
    """
    PyTorch version of 3-layer CNN
    """
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 7)
        self.relu1 = nn.LeakyReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size = 4)

        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 5)
        self.relu2 = nn.LeakyReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size = 3)

        self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 128, kernel_size = 3)
        self.relu3 = nn.LeakyReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size = 2)
        
        self.aap = nn.AdaptiveAvgPool2d(output_size=1)
        self.mlp = MLP(in_channels=128,hidden_channels=[3])

    def forward(self, x):

        x = x.float()
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)

        print(x.shape)
        x = self.aap(x)
        print(x.shape)
        x = x.flatten()
        print(x.shape)

        x = self.mlp(x)

        return x

In [144]:
conv_wb = i2p()
y = conv_wb.forward(resized_image)
print(y)

torch.Size([1, 128, 8, 8])
torch.Size([1, 128, 1, 1])
torch.Size([128])
tensor([ 330.7601,  -83.8726, -270.8345], grad_fn=<AddBackward0>)


In [3]:
with rawpy.imread(raw_filename) as raw:
    # get raw image data
    image = np.array(raw.raw_image, dtype=np.double)
    # subtract black levels and normalize to interval [0..1]
    black = np.reshape(np.array(raw.black_level_per_channel, dtype=np.double), (2, 2))
    black = np.tile(black, (image.shape[0]//2, image.shape[1]//2))
    image = (image - black) / (raw.white_level - black)
    # find the positions of the three (red, green and blue) or four base colors within the Bayer pattern
    n_colors = raw.num_colors
    colors = np.frombuffer(raw.color_desc, dtype=np.byte)
    pattern = np.array(raw.raw_pattern)
    index_0 = np.where(colors[pattern] == colors[0])
    index_1 = np.where(colors[pattern] == colors[1])
    index_2 = np.where(colors[pattern] == colors[2])
    index_3 = np.where(colors[pattern] == colors[3])
    # apply white balance, normalize white balance coefficients to the 2nd coefficient, which is ususally the coefficient for green
    wb_c = raw.camera_whitebalance 
    wb = np.zeros((2, 2), dtype=np.double) 
    wb[index_0] = wb_c[0] / wb_c[1]
    wb[index_1] = wb_c[1] / wb_c[1]
    wb[index_2] = wb_c[2] / wb_c[1]
    if n_colors == 4:
        wb[index_3] = wb_c[3] / wb_c[1]
    wb = np.tile(wb, (image.shape[0]//2, image.shape[1]//2))
    image_wb = np.clip(image * wb, 0, 1)
    # demosaic via downsampling
    image_demosaiced = np.empty((image_wb.shape[0]//2, image_wb.shape[1]//2, n_colors))
    if n_colors == 3:
        image_demosaiced[:, :, 0] = image_wb[index_0[0][0]::2, index_0[1][0]::2]
        image_demosaiced[:, :, 1]  = (image_wb[index_1[0][0]::2, index_1[1][0]::2] + image_wb[index_1[0][1]::2, index_1[1][1]::2]) / 2
        image_demosaiced[:, :, 2]  = image_wb[index_2[0][0]::2, index_2[1][0]::2]
    else: # n_colors == 4
        image_demosaiced[:, :, 0] = image_wb[index_0[0][0]::2, index_0[1][0]::2]
        image_demosaiced[:, :, 1] = image_wb[index_1[0][0]::2, index_1[1][0]::2]
        image_demosaiced[:, :, 2] = image_wb[index_2[0][0]::2, index_2[1][0]::2]
        image_demosaiced[:, :, 3] = image_wb[index_3[0][0]::2, index_3[1][0]::2]
    # convert to linear sRGB, calculate the matrix that transforms sRGB into the camera's primary color components and invert this matrix to perform the inverse transformation


NameError: name 'raw_filename' is not defined